In [1]:
import argparse
import pandas as pd
import file_management
import video_processing
import megadetector
import detectMD
import parse_results
import split
import classify

def main(image_dir, md_model, class_model, class_list):
    """
    This function is the main method to invoke all the sub functions
    to create a working directory for the image directory.

    Args:
        image_dir (str): The directory path containing the images or videos.
        model_file (str): The file path of the MegaDetector model.
        class_model (str): The file path of the classifier model.
        class_list (list): A list of classes or species for classification.

    Returns:
        pandas.DataFrame: Concatenated dataframe of animal and empty detections.
    """
    print("Setting up working directory...")
    # Create a working directory, build the file manifest from img_dir
    working_dir = file_management.WorkingDirectory(image_dir)
    files = file_management.build_file_manifest(
        image_dir, out_file=working_dir.filemanifest
        )
    print("Found %d files." % len(files))
    print("Processing videos...")
    # Video-processing to extract individual frames as images in to directory
    all_frames = video_processing.images_from_videos(
        files, out_dir=working_dir.vidfdir,
        out_file=working_dir.imageframes, parallel=True, frames=2
        )
    print("Running images and video frames through MegaDetector...")
    # Run all images and video frames through MegaDetector
    if (file_management.check_file(working_dir.mdresults)):
        detections = file_management.load_data(working_dir.mdresults)
    else:
        detector = megadetector.MegaDetector(md_model)
        md_results = detectMD.detect_MD_batch(detector, 
                                              all_frames["Frame"], 
                                              results=None)
        print("Converting MD JSON to pd dataframe and merging with manifest...")
        # Convert MD JSON to pandas dataframe, merge with manifest
        detections = parse_results.from_MD(md_results, 
                                           manifest=all_frames, 
                                           out_file=working_dir.mdresults)
    print("Extracting animal detections...")
    # Extract animal detections from the rest
    animals = split.getAnimals(detections)
    empty = split.getEmpty(detections)
    print("Predicting species of animal detections...")
    classifier = classify.load_classifier(class_model)
    # Use the classifier model to predict the species of animal detections
    pred_results = classify.predict_species(animals, classifier, batch=4)
    print("Applying predictions to animal detections...")
    animals = parse_results.from_classifier(
        animals, pred_results, class_list, out_file=working_dir.predictions
        )
    print("Concatenating animal and empty dataframes...")
    manifest = pd.concat([animals, empty])
    manifest.to_csv(working_dir.results)
    print("Final Results in " + working_dir.results)
    return manifest

In [2]:
results = main('/home/kyra/animl-py/examples/Southwest','/home/kyra/animl-py/models/md_v5a.0.0.pt',
    '/home/kyra/animl-py/models/southwest_v2.h5','/home/kyra/animl-py/models/southwest_classes.txt')

Setting up working directory...
Output file already exists and was last modified 2023-10-26 15:09:03.287700,                  would you like to load it? y/n: y
Found 13 files.
Processing videos...
Output file already exists and was last modified 2023-10-26 15:09:03.291700,                  would you like to load it? y/n: y
Running images and video frames through MegaDetector...


ModuleNotFoundError: No module named 'models'

In [4]:
detector = megadetector.MegaDetector('/home/kyra/animl-py/models/md_v5a.0.0.pt')

ModuleNotFoundError: No module named 'models'

In [4]:
classifier = classify.load_classifier('/home/kyra/animl-py/models/southwest_v2.h5')

Loaded model in 3.55 seconds


In [6]:
import torch

In [8]:
detector = torch.load('/home/kyra/animl-py/models/md_v5a.0.0.pt')

ModuleNotFoundError: No module named 'models'

In [4]:
import torch
test = torch.load("/mnt/machinelearning/Models/Cougar/Experiments/Full/40.pt")

In [5]:
type(test)

dict

In [6]:
def load_model(model_pt_path):
    if torch.cuda.is_available():
        print("GPU available")
        device = torch.device('cuda:0')
    else:
        device = 'cpu'
    checkpoint = torch.load(model_pt_path, map_location=device)

    for m in checkpoint['model'].modules():
        t = type(m)
        if t is torch.nn.Upsample and not hasattr(m, 'recompute_scale_factor'):
            m.recompute_scale_factor = None

    model = checkpoint['model'].float().fuse().eval()  # FP32 model
    return model

In [1]:
import torch

In [7]:
md = load_model('/home/kyra/animl-py/models/md_v5a.0.0.pt')

GPU available


Fusing layers... 
Model summary: 574 layers, 139990096 parameters, 0 gradients


In [1]:
import megadetector

In [3]:
md = megadetector.MegaDetector('/home/kyra/animl-py/models/md_v5a.0.0.pt')

Fusing layers... 
Model summary: 574 layers, 139990096 parameters, 0 gradients


Sending model to GPU


In [2]:
import wget

In [2]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=d1a902beab2e1916d8bf64f7a3faee8b731f611425d7e13e110ef2d2d981fe79
  Stored in directory: /home/kyra/.cache/pip/wheels/e1/e8/db/ebe4dcd7d7d11208c1e4e4ef246cea4fcc8d463c93405a6555
Successfully built wget


In [5]:
wget.download('https://sandiegozoo.box.com/shared/static/xj3496ii5hxtomf0s38axb1agn5u9up8.pt', out='../../models/md_v5a.0.0.pt')

-1 / unknown

'../../models/md_v5a.0.0.pt'